源码详细链接：https://github.com/imClumsyPanda/langchain-ChatGLM.git

In [ ]:
来自https://www.kaggle.com/code/xuyouqian/chatglm-6b-langchain

In [ ]:
!pip install -q langchain 
!pip install -q sentence_transformers
!pip install -q faiss-cpu 
# 或者 faiss-gpu  文档参考https://faiss.ai/
# https://huggingface.co/shibing624/text2vec-base-chinese

### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
1. [环境依赖](#1)
2. [技术原理](#2)
3. [langchain使用](#3)
    - 3.1 [加载文档](#3.1)
    - 3.2 [文档分句](#3.2)
    - 3.3 [创建自定义向量数据库](#3.3)
    - 3.4 [保存向量数据库](#3.4)
    - 3.5 [知识检索](#3.5)
4. [ChatGLM-6B调用](https://github.com/THUDM/ChatGLM-6B)

# 1.环境依赖 <a id=1> [↑](#top)

```
langchain==0.0.181
sentence_transformers==2.2.2
faiss-cpu==1.7.4
```

# 2. 技术原理  <a id=2> [↑](#top)

项目实现原理如下图所示，过程包括加载文件 -> 读取文本 -> 文本分割 -> 文本向量化 -> 问句向量化 -> 在文本向量中匹配出与问句向量最相似的top k个 -> 匹配出的文本作为上下文和问题一起添加到 prompt 中 -> 提交给 LLM 生成回答。


![](https://mmbiz.qpic.cn/mmbiz_png/v8icavvN2YeKE1ibSBxpGMVzDBNmsRzD51Bt8A7KLtibHTK1gDRm5YMofOJXJnR6AD0uRoxpnv1qiaffSgkDpxkyiaw/640?wx_fmt=png&wxfrom=5&wx_lazy=1&wx_co=1)

从上面就能看出，其核心技术就是向量 embedding，将用户知识库内容经过 embedding 存入向量知识库，然后用户每一次提问也会经过 embedding，利用向量相关性算法（例如余弦算法）找到最匹配的几个知识库片段，将这些知识库片段作为上下文，与用户问题一起作为 prompt 提交给 LLM 回答，很好理解吧。一个典型的 prompt 模板如下：

"""
已知信息：
{context} 

根据上述已知信息，简洁和专业的来回答用户的问题。如果无法从中得到答案，请说 “根据已知信息无法回答该问题” 或 “没有提供足够的相关信息”，不允许在答案中添加编造成分，答案请使用中文。 
问题是：{question}
"""

# 3.langchain使用 <a id=3> [↑](#top)

### 3.1  加载文档  <a id=3.1> [↑](#top)

文档全文如下


```
"cotext":"问题:一位女性患者因上腹部疼痛、消化不良等症状就诊，经内窥镜检查发现幽门螺旋杆菌阳性并伴有胃癌，需要诊断和治疗。回答: " "target ": "该患者需要进行幽门螺杆菌根除治疗，同时进一步进行组织学检查（胃肿块活检)、影像学检查（腹/盆腔CT、PET/CT扫描）等，以确诊胃癌。治疗方面可能需要进行放化疗治疗措施，如D2淋巴结清扫术、一线化疗药物表柔比星、氟尿嘘啶、紫杉醇、卡培他滨等，辅助幽门螺杆菌根除治疗。同时还要注意并发症的预防和处理，如穿孔、出血等。
"cotext":"问题:王先生，60岁，有长期胃黏膜炎症的病史，最近出现胃肠道症状加重，甚至出现消瘦、贫血等情况。回答: " "target" "应考虑幽门螺杆菌感染引起的长期胃黏膜炎症、胃癌等并发症，建议进行幽门螺杆菌检测和胃镜检查，如确诊应进行相应的治疗和随访。"""cotext":"问题:一位孕妇在妊娠第三个月时被检查出贫血，她应该注意哪些方面来预防该病症加重?回答: " "target": "孕妇在日常生活中应保证足够的摄入含铁质的食物，如红肉、黑木耳等。
""cotext":"问题:李先生，45岁，去医院就诊，发现肝内胆管癌，询问能否手术治疗﹖回答: " "target":"肝内胆管癌可以进行肝部分切除术进行治疗。"""cotext":"问题:一名35岁男性患者，近期出现中上腹持续性疼痛、压痛、多梦、腹泻等症状，饭后加剧。辅助检查中，组织学检查发现胃黏膜损害。请问该患者可能患有什么疾病，并发症是什么?回答:"target":"该患者可能患有十二指肠胃反流及胆汁反流性胃炎
```

In [ ]:
from langchain.document_loaders import TextLoader
# autodetect_encoding=True是一个参数，用于在创建TextLoader对象时指定是否自动检测文本文件的编码。
# 在您提供的代码中，您在加载state_of_the_union_ch.txt文件时设置了autodetect_encoding=True
loader = TextLoader('/kaggle/input/langchaindata/state_of_the_union_ch.txt',autodetect_encoding=True)


In [ ]:
loader

### 3.2 文档分句 <a id=3.2> [↑](#top)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
import re
from typing import List
SENTENCE_SIZE = 100

In [ ]:
# split_text 是自定义的分句方法  可以根据自己的需求定义
class ChineseTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, sentence_size: int = SENTENCE_SIZE, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf
        self.sentence_size = sentence_size

    def split_text1(self, text: str) -> List[str]:
        if self.pdf:
            text = re.sub(r"\n{3,}", "\n", text)
            text = re.sub('\s', ' ', text)
            text = text.replace("\n\n", "")
        sent_sep_pattern = re.compile('([﹒﹔﹖﹗．。！？]["’”」』]{0,2}|(?=["‘“「『]{1,2}|$))')  # del ：；
        sent_list = []
        for ele in sent_sep_pattern.split(text):
            if sent_sep_pattern.match(ele) and sent_list:
                sent_list[-1] += ele
            elif ele:
                sent_list.append(ele)
        return sent_list

    def split_text(self, text: str) -> List[str]:   ##此处需要进一步优化逻辑
        if self.pdf:
            text = re.sub(r"\n{3,}", r"\n", text)
            text = re.sub('\s', " ", text)
            text = re.sub("\n\n", "", text)

        text = re.sub(r'([;；.!?。！？\?])([^”’])', r"\1\n\2", text)  # 单字符断句符
        text = re.sub(r'(\.{6})([^"’”」』])', r"\1\n\2", text)  # 英文省略号
        text = re.sub(r'(\…{2})([^"’”」』])', r"\1\n\2", text)  # 中文省略号
        text = re.sub(r'([;；!?。！？\?]["’”」』]{0,2})([^;；!?，。！？\?])', r'\1\n\2', text)
        # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
        text = text.rstrip()  # 段尾如果有多余的\n就去掉它
        # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
        ls = [i for i in text.split("\n") if i]
        for ele in ls:
            if len(ele) > self.sentence_size:
                ele1 = re.sub(r'([,，.]["’”」』]{0,2})([^,，.])', r'\1\n\2', ele)
                ele1_ls = ele1.split("\n")
                for ele_ele1 in ele1_ls:
                    if len(ele_ele1) > self.sentence_size:
                        ele_ele2 = re.sub(r'([\n]{1,}| {2,}["’”」』]{0,2})([^\s])', r'\1\n\2', ele_ele1)
                        ele2_ls = ele_ele2.split("\n")
                        for ele_ele2 in ele2_ls:
                            if len(ele_ele2) > self.sentence_size:
                                ele_ele3 = re.sub('( ["’”」』]{0,2})([^ ])', r'\1\n\2', ele_ele2)
                                ele2_id = ele2_ls.index(ele_ele2)
                                ele2_ls = ele2_ls[:ele2_id] + [i for i in ele_ele3.split("\n") if i] + ele2_ls[
                                                                                                       ele2_id + 1:]
                        ele_id = ele1_ls.index(ele_ele1)
                        ele1_ls = ele1_ls[:ele_id] + [i for i in ele2_ls if i] + ele1_ls[ele_id + 1:]

                id = ls.index(ele)
                ls = ls[:id] + [i for i in ele1_ls if i] + ls[id + 1:]
        return ls
    
    def split_text(self, text: str) -> List[str]:   ##此处需要进一步优化逻辑
        if self.pdf:
            text = re.sub(r"\n{3,}", r"\n", text)
            text = re.sub('\s', " ", text)
            text = re.sub("\n\n", "", text)
        ls = [i for i in text.split("\n") if i]
        
        return ls 

In [ ]:
loader = TextLoader('/kaggle/input/langchaindata/yixuezhishiku.txt',autodetect_encoding=True)

In [ ]:
sentence_size = 500
textsplitter = ChineseTextSplitter(pdf=False, sentence_size=sentence_size)
docs = loader.load_and_split(textsplitter)

In [ ]:
docs

### 3.3 创建向量数据库 <a id=3.3> [↑](#top)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='GanymedeNil/text2vec-large-chinese')
embeddings

In [ ]:
sentences = ['如何更换花呗绑定银行卡', '花呗更改绑定银行卡']
vectors = embeddings.embed_documents(sentences)
print(len(vectors[0]))

In [ ]:
vector_store = FAISS.from_documents(docs,  embeddings)

### 3.4 保存和加载本地向量数据库  <a id=3.4> [↑](#top)

保存向量数据库

In [ ]:
vector_store.save_local('vector_store_path')

加载向量数据库

In [ ]:
local_vector_store = FAISS.load_local('vector_store_path', embeddings)

添加新的向量

In [ ]:
loader = TextLoader('/kaggle/input/langchaindata/state_of_the_union_ch.txt',autodetect_encoding=True)
new_docs = loader.load_and_split(textsplitter)
local_vector_store.add_documents(new_docs)

### 3.5 知识检索 <a id=3.5> [↑](#top)

In [ ]:
# 知识检索内容相关度 Score, 数值范围约为0-1100，如果为0，则不生效，经测试设置为小于500时，匹配结果更精准
local_vector_store.score_threshold = 499
related_docs_with_score = local_vector_store.similarity_search_with_score("肚子痛", k=5)
related_docs_with_score

In [ ]:
context = "\n".join([doc[0].page_content for doc in related_docs_with_score])
context

In [ ]:
prompt_template = """已知信息：
{context} 

根据上述已知信息，简洁和专业的来回答用户的问题。如果无法从中得到答案，请说 “根据已知信息无法回答该问题” 或 “没有提供足够的相关信息”，不允许在答案中添加编造成分，答案请使用中文。 问题是：{question}"""

In [ ]:
query = '肚子痛'
prompt = prompt_template.replace("{question}", query).replace("{context}", context)
prompt